In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import praw
import os
import re
import nltk
import time
import random
import emoji
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models
import string
import re
import emoji
import warnings
warnings.filterwarnings("ignore")

from spacy.lang.en.stop_words import STOP_WORDS as stopwords

from string import punctuation
from collections import Counter, defaultdict
from wordcloud import WordCloud
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer, PorterStemmer 
from top2vec import Top2Vec



[nltk_data] Downloading package stopwords to C:\Users\Luis
[nltk_data]     Perez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('pos_pre_processed.csv')

In [73]:
# Drop rows with NaN values in categories column
df = df.dropna(subset=['categories'])

# Keep only rows where categories contain 'restaurants' or 'food'
df = df[df['categories'].str.contains('restaurants|food', case=False, regex=True)]

# Get the length of the resulting dataframe
length = len(df)

print(f"Length of dataset: {length}")

Length of dataset: 173578


In [75]:
rev_list = df['text_processed'].tolist()
rev_list

rev_list = [str(elem) for elem in rev_list]

In [76]:
model = Top2Vec(rev_list)

topic_sizes, topic_nums = model.get_topic_sizes()
print(topic_sizes)

print(topic_nums)

topic_words, word_scores, topic_nums = model.get_topics()

for words, scores , num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f"Words: {words}")
    
# Get the top words for each new topic
num_topics = len(topic_mapping)
new_topic_words, _, _ = model.get_topics(num_topics=num_topics, reduced=True)

# Print the new topics and the top 10 words for each topic
for i, words in enumerate(new_topic_words):
    print(f"New Topic {topic_nums[i]}: {words[:20]}")    

2023-04-02 16:32:55,994 - top2vec - INFO - Pre-processing documents for training
2023-04-02 16:33:50,062 - top2vec - INFO - Creating joint document/word embedding
2023-04-02 16:47:16,156 - top2vec - INFO - Creating lower dimension embedding of documents
2023-04-02 16:49:01,904 - top2vec - INFO - Finding dense areas of documents
2023-04-02 16:49:23,762 - top2vec - INFO - Finding topics


[2186 2091 2015 ...   28   28   26]
[   0    1    2 ... 1149 1150 1151]
0
Words: ['barista' 'latte' 'espresso' 'americano' 'roaster' 'handlebar' 'coffee'
 'mocha' 'starbuck' 'oat' 'verve' 'caffeine' 'roasting' 'cappuccino'
 'chai' 'laptop' 'caje' 'dune' 'wifi' 'macchiato' 'nitro' 'cyclist' 'iced'
 'foam' 'outlet' 'matcha' 'milk' 'gibraltar' 'brew' 'llama' 'study'
 'decaf' 'peet' 'sweetener' 'rebar' 'mug' 'expresso' 'lech' 'press' 'shop'
 'merchandise' 'ping' 'sip' 'croissant' 'aaron' 'ceramic' 'hipster' 'dart'
 'couch' 'bicycle']
1
Words: ['acknowledge' 'ignore' 'apologize' 'rude' 'apology' 'minute'
 'unprofessional' 'hostess' 'flag' 'politely' 'unacceptable' 'manager'
 'rudely' 'annoyed' 'horrible' 'attitude' 'response' 'min' 'reply' 'poor'
 'she' 'upset' 'proceed' 'ask' 'waiter' 'management' 'bill' 'interrupt'
 'contact' 'invisible' 'finally' 'waitress' 'understaffed' 'busser'
 'behavior' 'refuse' 'act' 'nobody' 'manger' 'confront' 'receive' 'excuse'
 'angry' 'rudeness' 'apologetic' 

ValueError: Hierarchical topic reduction has not been performed.

In [77]:
print(topic_nums)



[   0    1    2 ... 1149 1150 1151]


In [30]:
# generate  topics
n_topics =model.get_num_topics()

# get the top 10 words for each topic
topic_words, word_scores, topic_nums = model.get_topics(num_topics=n_topics)

# loop through each topic and print the top words
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(f"Topic {num}: {words}")

Topic 0: ['rude' 'unprofessional' 'rudely' 'apology' 'apologize' 'disrespectful'
 'ignore' 'accuse' 'unacceptable' 'rudeness' 'manager' 'disrespect'
 'acknowledge' 'condescend' 'threaten' 'dismissive' 'defensive' 'refund'
 'attitude' 'behavior' 'yell' 'politely' 'confront' 'response' 'hostile'
 'voicemail' 'angry' 'excuse' 'upset' 'refuse' 'racist' 'unfriendly'
 'horrible' 'unwelcome' 'harass' 'interrupt' 'incompetent' 'reply'
 'blatantly' 'management' 'act' 'liar' 'insult' 'poor' 'arrogant' 'manger'
 'hostess' 'terrible' 'inappropriate' 'audacity']
Topic 1: ['continental' 'kitchenette' 'bedding' 'brisas' 'motel' 'innkeeper' 'inn'
 'amenity' 'jacuzzi' 'cheshire' 'suite' 'castillo' 'upham' 'dvd'
 'housekeeping' 'toiletry' 'tub' 'lobby' 'expedia' 'cruiser' 'bungalow'
 'whirlpool' 'hotel' 'bed' 'cereal' 'bathtub' 'luggage' 'desk' 'dvds'
 'robe' 'casita' 'bedroom' 'appoint' 'wayfarer' 'furnishing'
 'accommodation' 'shutter' 'slipper' 'rooftop' 'hostel' 'airstream' 'stay'
 'shower' 'cable' 

In [93]:
topic_mapping = model.hierarchical_topic_reduction(num_topics=20)

In [94]:
# Get the top words for each new topic
num_topics = len(topic_mapping)
new_topic_words, _, _ = model.get_topics(num_topics=num_topics, reduced=True)

# Print the new topics and the top 10 words for each topic
for i, words in enumerate(new_topic_words):
    print(f"New Topic {topic_nums[i]}: {words[:25]}")

New Topic 0: ['toma' 'risotto' 'fava' 'ragout' 'umbra' 'rigatoni' 'pudde' 'sweetbread'
 'bouchon' 'veal' 'lasagna' 'raviolis' 'cannoli' 'tagliatelle' 'branzino'
 'gnocchi' 'dario' 'ravioli' 'bucatini' 'alla' 'burrata' 'polenta'
 'vittoria' 'confit' 'cous']
New Topic 1: ['rude' 'unprofessional' 'unacceptable' 'apologize' 'manager' 'response'
 'apology' 'acknowledge' 'attitude' 'rudely' 'politely' 'ignore'
 'management' 'behavior' 'refund' 'disrespectful' 'upset' 'confront'
 'refuse' 'reply' 'accuse' 'horrible' 'racist' 'yell' 'angry']
New Topic 2: ['picadillo' 'jamal' 'macro' 'baklava' 'cadiz' 'messina' 'polenta' 'verde'
 'toma' 'blake' 'convivo' 'breanna' 'remy' 'greenhouse' 'oku' 'empanadas'
 'loquita' 'tamira' 'philippe' 'zaytoon' 'blta' 'jackfruit' 'foxtail'
 'tomate' 'cask']
New Topic 3: ['pepperoni' 'pizza' 'pastrami' 'crust' 'plt' 'margherita' 'calzone'
 'rockfire' 'buffalo' 'panini' 'foccacia' 'turkey' 'provolone' 'pesto'
 'focaccia' 'burger' 'sandwich' 'hamburger' 'guru' 'parma

In [ ]:
rev_list = df['text_processed'].tolist()
rev_list

In [ ]:
rev_list = [str(elem) for elem in rev_list]

In [ ]:
customer_service = ['clean','dirty','spotless','sanitary','shiny','messy','environment','atmosphere','vibe','mood','ambience', 'decoration',
                       'relax', 'architecture']

service_keyword = ['service','customer','rude','mean','rushed','friendly', 'patient', 'server', 'waiter', 'waitress','timely',
                   'slow','fast','quickly','forever','refund','tip', 'wait']
price_keyword = ['expensive','cheap','price','cost','dollar','dollars', 'money', 'pricy','ripoff','affordable', 'value', 'discount', 'bill']




















# reduce by topic hierarchy 

from 1.2k to 50 with 100 word bank

In [39]:
#topic_mapping = model.hierarchical_topic_reduction(num_topics=50)
# Get the top words for each new topic
num_topics = len(topic_mapping)
new_topic_words, _, _ = model.get_topics(num_topics=num_topics, reduced=True)

# Print the new topics and the top 50 words for each topic
for i, words in enumerate(new_topic_words):
    print(f"New Topic {topic_nums[i]}: {words[:100]}")

New Topic 0: ['rude' 'unprofessional' 'rudely' 'apology' 'disrespectful' 'apologize'
 'ignore' 'unacceptable' 'accuse' 'rudeness' 'manager' 'disrespect'
 'acknowledge' 'dismissive' 'threaten' 'yell' 'condescend' 'confront'
 'defensive' 'attitude' 'refund' 'hostile' 'politely' 'angry' 'behavior'
 'excuse' 'upset' 'refuse' 'unwelcome' 'response' 'voicemail' 'harass'
 'unfriendly' 'horrible' 'interrupt' 'racist' 'blatantly' 'management'
 'incompetent' 'act' 'poor' 'liar' 'insult' 'manger' 'piss' 'arrogant'
 'hostess' 'audacity' 'police' 'terrible']
New Topic 1: ['benedict' 'hash' 'scramble' 'omelette' 'tugboat' 'challah' 'biscuit'
 'corned' 'omelet' 'pancake' 'eggs' 'hollandaise' 'poach' 'beignet'
 'gravy' 'florentine' 'breakfast' 'cristo' 'tupelo' 'brunch' 'pancakes'
 'kahlua' 'huevos' 'yolo' 'egg' 'hashbrown' 'toast' 'rancheros' 'grit'
 'patrol' 'ranchero' 'oj' 'junction' 'latte' 'scrambled' 'chilaquile'
 'poached' 'ihop' 'scone' 'sambo' 'norwegian' 'muffin' 'omlette'
 'andouille' 'oatm

# search by kerwords

In [97]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["price"], num_docs=100)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 112992, Score: 0.4732450246810913
-----------
super essen geschmacklich wie in italien aber in dieser zu gehobenen preisen
-----------

Document: 120525, Score: 0.42710453271865845
-----------
wonderful food and service our place in santa barbara
-----------

Document: 134120, Score: 0.40768152475357056
-----------
the food at this place be very good we have a great lunch with excellent customer service
-----------

Document: 27769, Score: 0.40002691745758057
-----------
I think the food be worth the price a lot of restaurant in sb be much well than here
-----------

Document: 149966, Score: 0.38446131348609924
-----------
food be great service be good christianna be amazing and I have to ask for pretty much anything would definitely come back here next time in town
-----------

Document: 71955, Score: 0.3647647202014923
-----------
I love this place price be good fish be fresh definitely my go to place for a nice fish dinner for a decent price
-----------

Document: 71303, S

In [98]:
words, word_scores = model.similar_words(keywords=["price", ], keywords_neg=[], num_words=10)
for word, score in zip(words, word_scores):
    print(f"'{word}',")
    #print(f"{word} {score}")

'reasonable',
'expensive',
'quality',
'worth',
'high',
'cheap',
'decent',
'than',
'probably',
'good',
